In [ ]:
import json
from collections import defaultdict
import math

# Global constants
MAX_RECURSION_DEPTH = 6  # Default max recursion depth

def load_data():
    """Load the Satisfactory data from the JSON file."""
    with open("raw_data/data.json", 'r', encoding='utf-8') as file:
        return json.load(file)

def get_item_recipes(data, item_id):
    """Get all recipes that produce the given item."""
    recipes = []

    # First, validate that the item actually exists
    if item_id not in data.get('items', {}) and item_id not in data.get('resources', {}):
        print(f"Warning: Item '{item_id}' does not exist in the data!")
        return []

    if item_id in data.get('resources', {}):
        print(f"Item '{item_id}' is a resource, not a recipe product.")
        return

    # Find recipes that produce this item
    for recipe_id, recipe_data in data['recipes'].items():
        if not recipe_data.get('products'):
            continue

        for product in recipe_data.get('products', []):
            if not isinstance(product, dict):
                continue

            product_item = product.get('item')
            if product_item == item_id:
                for ingredient in recipe_data.get('ingredients', []):
                    if not isinstance(ingredient, dict):
                        continue
                    ingredient_item = ingredient.get('item')
                    print(f"Found recipe {recipe_id} for item {item_id} with ingredient {ingredient_item}")

                    for ingredient_recipe in get_item_recipes(data, ingredient_item):
                        yield recipe_id
                        print(f"Recursively found recipe {ingredient_recipe} for ingredient {ingredient_item}")
                        yield ingredient_recipe
                break



def display_recipes(data, recipes):
    """Display the recipes in a readable format."""
    if not recipes:
        print("No recipes found.")
        return

    print("Recipes:")
    print("=" * 80)
    for recipe_id in recipes:
        recipe_data = data['recipes'].get(recipe_id, {})
        recipe_name = recipe_data.get('name', recipe_id)
        print(f"Recipe ID: {recipe_id} - {recipe_name}")
        print(f"  Machine: {recipe_data.get('machine', 'Unknown')}")
        print(f"  Time: {recipe_data.get('time', 1.0)} seconds")
        print(f"  Power Usage: {recipe_data.get('power_use', 0)} MW")
        print("  Products:")
        for product in recipe_data.get('products', []):
            item = product.get('item')
            amount = product.get('amount', 1)
            print(f"    - {item} x{amount}")
        print("  Ingredients:")
        for ingredient in recipe_data.get('ingredients', []):
            item = ingredient.get('item')
            amount = ingredient.get('amount', 1)
            print(f"    - {item} x{amount}")
        print()

def is_valid_item(data, item_id):
    """Check if an item ID is valid."""
    if item_id not in data.get('items', {}) and item_id not in data.get('resources', {}):
        print(f"Warning: Item '{item_id}' does not exist in the data!")
        return False
    return True

def is_resource(data, item_id):
    """Check if an item is a resource."""
    if not is_valid_item(data, item_id):
        return False
    return item_id in data.get('resources', {})

def is_crafted(data, item_id):
    """Check if an item is crafted by a recipe."""
    if not is_valid_item(data, item_id):
        return False
    if is_resource(data, item_id):
        return False
    for recipe_id, recipe_data in data['recipes'].items():
        if not recipe_data.get('products'):
            continue
        for product in recipe_data.get('products', []):
            if not isinstance(product, dict):
                continue
            product_item = product.get('item')
            if product_item == item_id:
                return True
    return False


def get_item_recipes(data,item_id):
    """Get all recipes that produce the given item."""
    recipes = []

    # First, validate that the item actually exists
    if item_id not in data.get('items', {}) and item_id not in data.get('resources', {}):
        print(f"Warning: Item '{item_id}' does not exist in the data!")
        return []

    if item_id in data.get('resources', {}):
        print(f"Item '{item_id}' is a resource, not a recipe product.")
        return []

    # Find recipes that produce this item
    for recipe_id, recipe_data in data['recipes'].items():
        if not recipe_data.get('products'):
            continue

        for product in recipe_data.get('products', []):
            if not isinstance(product, dict):
                continue

            product_item = product.get('item')
            if product_item == item_id:
                recipes.append(recipe_id)
                break

    return recipes

def display_recipe(recipe):
    print(f"{recipe.get('name', 'Unknown')}")
    #print(f"  Machine: {recipe.get('machine', 'Unknown')}, Time: {recipe.get('time', 1.0)} seconds")
    print("  Products:")
    for product in recipe.get('products', []):
        item = product.get('item')
        amount = product.get('amount', 1)
        print(f"    - {item} x{amount}")
    print("  Ingredients:")
    for ingredient in recipe.get('ingredients', []):
        item = ingredient.get('item')
        amount = ingredient.get('amount', 1)
        print(f"    - {item} x{amount}")

def build_recipe_tree(data, item_id, recipe_branch=None, recursion_level=0):
    if recursion_level > MAX_RECURSION_DEPTH:
        print(f"Max recursion depth reached for item {item_id}. Stopping recursion.")
        return False
    if not is_valid_item(data, item_id):
        return False
    if is_resource(data, item_id):
        return True
    for recipe_id in get_item_recipes(data, item_id):
        # Get the recipe for the recipe id
        recipe_data = data['recipes'].get(recipe_id, {})
        if "package" in recipe_id.lower():
            continue
        for ingredient in recipe_data.get('ingredients', []):
            if not isinstance(ingredient, dict):
                continue
            ingredient_item = ingredient.get('item')
            print(f"*** Look for ingredient {ingredient_item} in recipe {recipe_id} at level {recursion_level}***")
            if "Unpackaged" in ingredient_item.lower():
                continue
            #print(f"Found ingredient {ingredient_item} in recipe {recipe_id}")
            # Recursively build the tree for the ingredient
            if not build_recipe_tree(data, ingredient_item, recipe_branch, recursion_level=recursion_level + 1):
                break
            else:
                display_recipe(recipe_data)


def get_recipe(data, item_id, branches=[], branch={}, recursion_level=0):
    if recursion_level > MAX_RECURSION_DEPTH:
        print(f"Max recursion depth reached for item {item_id}. Stopping recursion.")
        return False
    if not is_valid_item(data, item_id):
        return False
    if is_resource(data, item_id):
        return True
    for recipe_id in get_item_recipes(data, item_id):
        # Get the recipe for the recipe id
        recipe_data = data['recipes'].get(recipe_id, {})
        if "package" in recipe_id.lower():
            continue
        branch = recipe_data
        for ingredient in recipe_data.get('ingredients', []):
            if not isinstance(ingredient, dict):
                continue
            ingredient_item = ingredient.get('item')
            print(f"*** Look for ingredient {ingredient_item} in recipe {recipe_id} at level {recursion_level}***")
            if "Unpackage" in ingredient_item.lower():
                continue
            #print(f"Found ingredient {ingredient_item} in recipe {recipe_id}")
            # Recursively build the tree for the ingredient
            if not get_recipe(data, ingredient_item, branches, branch, recursion_level=recursion_level + 1):
                break
            else:
                display_recipe(recipe_data)


In [2]:
data = load_data()
build_recipe_tree(data, 'Desc_SteelPipe_C')

*** Look for ingredient Desc_SteelIngot_C in recipe Recipe_SteelPipe_C at level 0***
*** Look for ingredient Desc_OreIron_C in recipe Recipe_IngotSteel_C at level 1***
Steel Ingot
  Products:
    - Desc_SteelIngot_C x3
  Ingredients:
    - Desc_OreIron_C x3
    - Desc_Coal_C x3
*** Look for ingredient Desc_Coal_C in recipe Recipe_IngotSteel_C at level 1***
Steel Ingot
  Products:
    - Desc_SteelIngot_C x3
  Ingredients:
    - Desc_OreIron_C x3
    - Desc_Coal_C x3
*** Look for ingredient Desc_OreIron_C in recipe Recipe_Alternate_CokeSteelIngot_C at level 1***
Alternate: Coke Steel Ingot
  Products:
    - Desc_SteelIngot_C x20
  Ingredients:
    - Desc_OreIron_C x15
    - Desc_PetroleumCoke_C x15
*** Look for ingredient Desc_PetroleumCoke_C in recipe Recipe_Alternate_CokeSteelIngot_C at level 1***
*** Look for ingredient Desc_HeavyOilResidue_C in recipe Recipe_PetroleumCoke_C at level 2***
*** Look for ingredient Desc_LiquidOil_C in recipe Recipe_Plastic_C at level 3***
Plastic
  Produ